# Introduction to `linopy`

:::{note}
This material is in part adapted from the following resources:
- [Linopy Tutorial](https://linopy.readthedocs.io/en/latest/index.html)
- [PyPSA simple electricity market examples](https://pypsa.readthedocs.io/en/latest/examples/simple-electricity-market-examples.html)
:::



[Linopy](https://linopy.readthedocs.io/en/latest/index.html)  is an open-source framework for formulating, solving, and analyzing optimization problems with Python.

With Linopy, you can create an optimization model within Python that consists of decision variables, constraints, and an optimization objective. You can then solve these instances using a variety of commerci and open-source solvers (specialised software).

[Linopy](https://linopy.readthedocs.io/en/latest/index.html) supports a wide range of problem types, including:

- **Linear programming**
- Quadratic programmingng
- Nonlinear programming
- Mixed-integer linear programming
- Mixed-integer quadratic programming
- Mixed-integer nonlinear programming
- Stochastic programming
- Generalized disjunctive programming
- Differential algebraic equations
- Bilevel programmin- Mathematical programs with equilibrium constraints


:::{note}
Documentation for this package is available at https://linopy.readthedocs.io/en/latest/index.html.
:::.
:::

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install pandas pyomo highspy
```
:::

### Initializing a `Model`

The Model class in Linopy is a fundamental part of the library. It serves as a container for all the relevant data associated with a linear optimization problem. This includes variables, constraints, and the objective function.

In [1]:
import pandas as pd
import numpy as np

import xarray as xr

In [2]:
from linopy import Model  # , Variable, Objective#, Constraint, Sense

In [3]:
# Create a linopy Problem
m = Model()

This creates a new Model object, which you can then use to define your optimization problem.



### Adding variables

Variables in a linear optimization problem represent the decision variables. A variable can always be assigned with a lower and an upper bound. In our case, both `x` and `y` have a lower bound of zero (default is unbouded). In Linopy, you can add variables to a Model using the `add_variables` method:

In [4]:
# Create Variables

x = m.add_variables(lower=0, name="x")
y = m.add_variables(lower=0, name="y");
# m.x = Variable(domain=(0, float('inf')))
# m.y = Variable()

`x` and `y` are linopy variables of the class `linopy.Variable`. Each of them contain all relevant information that define it. The `name` parameter is optional but can be useful for referencing the variables later.

In [5]:
y

Variable
--------
y ∈ [0, inf]

## Adding the Objective 

The objective function defines what you want to optimize. You can set the objective function of a Model in Linopy using the add_objective method. For our example that would be

In [6]:
# Create Objective
m.add_objective(-4 * x - 3 * y)
# m.obj = Objective(4 * m.x + 3 * m.y, sense=Sense.MAXIMIZE)

LinearExpression
----------------
-4 x - 3 y

Since both `x` and `y` are scalar variables (meaning they don't have any dimensions), their underlying data contain only one optimization variable each.  

### Adding Constraints

Constraints define the feasible region of the optimization problem. They consist of the left hand side (lhs) and the right hand side (rhs). The first constraint that we want to write down is 
$x <= 4$, which we write out exactly in the mathematical way.

We assign it by calling the function `m.add_constraints`.

In [7]:
# Create Constraints

m.add_constraints(x <= 4)
m.add_constraints(2 * x + y <= 10)
m.add_constraints(x + y <= 10)

Constraint `con2`
-----------------
+1 x + 1 y ≤ 10

## Solving the Model

Once you've defined your Model with variables, constraints, and an objective function, you can solve it using the `solve` method:

In [8]:
# Solve the Problem
m.solve()
# m.solve(solver="cbc")  # You can choose a different solver like 'glpk', 'gurobi', etc.

Restricted license - for non-production use only - expires 2024-10-28
Read LP format model from file C:\Users\huang\AppData\Local\Temp\linopy-problem-c1f_f7db.lp
Reading time = 0.00 seconds
obj: 3 rows, 2 columns, 5 nonzeros
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 2 columns and 5 nonzeros
Model fingerprint: 0x36ba073a
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 1e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -3.0000000e+01   0.000000e+00   0.000000e+00      0s
       0   -3.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 ite

('ok', 'optimal')

In [9]:
x.solution.values

array(0.)

In [10]:
y.solution.values

array(10.)

In [11]:
# Print Solver Status
print("Solver Status:", m.status)

Solver Status: ok


While we can read from the message above that our problem was solved successfully, we can also formally check by accessing the reported status in the Linopy "Model()" object.

In [12]:
assert m.status == "ok"

## Electricity Market Examples

### Single bidding zone, single period

We want to minimise operational cost of an example electricity system representing South Africa subject to generator limits and meeting the load:

\begin{equation}
    \min_{g_s} \sum_s o_s g_s
  \end{equation}
  such that
  \begin{align}
    g_s &\leq G_s \\
    g_s &\geq 0 \\
    \sum_s g_s &= d
  \end{align}

We are given the following information on the South African electricity system:

Marginal costs in EUR/MWh

In [13]:
marginal_costs = {
    "Wind": 0,
    "Coal": 30,
    "Gas": 60,
    "Oil": 80,
}

Power plant capacities in MW

In [14]:
capacities = {"Coal": 35000, "Wind": 3000, "Gas": 8000, "Oil": 2000}

Inelastic demand in MW

In [15]:
load = 42000

In [16]:
m = Model()

In [17]:
fuel_type = pd.Index(capacities.keys(), name='fuel_type')
fuel_type

Index(['Coal', 'Wind', 'Gas', 'Oil'], dtype='object', name='fuel_type')

In [18]:
capacities_series = pd.Series(capacities.values(),index=capacities.keys())
capacities_series

Coal    35000
Wind     3000
Gas      8000
Oil      2000
dtype: int64

In [19]:
g = m.add_variables(lower=0, upper=capacities_series, coords=[fuel_type], name='g')
g

Variable (fuel_type: 4)
-----------------------
[Coal]: g[Coal] ∈ [0, 3.5e+04]
[Wind]: g[Wind] ∈ [0, 3000]
[Gas]: g[Gas] ∈ [0, 8000]
[Oil]: g[Oil] ∈ [0, 2000]

In [20]:
marginal_costs_new = {k : marginal_costs[k] for k in capacities.keys()}
marginal_costs_new

{'Coal': 30, 'Wind': 0, 'Gas': 60, 'Oil': 80}

In [21]:
marginal_costs_series= pd.Series(marginal_costs_new.values(),index=marginal_costs_new.keys())
marginal_costs_series

Coal    30
Wind     0
Gas     60
Oil     80
dtype: int64

In [22]:
g.sum() == load

Constraint (unassigned)
-----------------------
+1 g[Coal] + 1 g[Wind] + 1 g[Gas] + 1 g[Oil] = 42000

It always helps to write out the constraints before adding them to the model. Since they look good, let’s assign them.

In [23]:
m.add_constraints(g.sum() == load, name='total-generation-constraint')

Constraint `total-generation-constraint`
----------------------------------------
+1 g[Coal] + 1 g[Wind] + 1 g[Gas] + 1 g[Oil] = 42000

In [24]:
obj = (g*marginal_costs_series).sum()
m.add_objective(obj)

LinearExpression
----------------
+30 g[Coal] + 0 g[Wind] + 60 g[Gas] + 80 g[Oil]

In [25]:
m.solve()

Restricted license - for non-production use only - expires 2024-10-28
Read LP format model from file C:\Users\huang\AppData\Local\Temp\linopy-problem-unwfqqyd.lp
Reading time = 0.00 seconds
obj: 1 rows, 4 columns, 4 nonzeros
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 4 columns and 4 nonzeros
Model fingerprint: 0x9b6e654b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 8e+01]
  Bounds range     [2e+03, 4e+04]
  RHS range        [4e+04, 4e+04]
Presolve removed 1 rows and 4 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2900000e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.290

('ok', 'optimal')

In [26]:
g.solution.fuel_type.values

array(['Coal', 'Wind', 'Gas', 'Oil'], dtype=object)

In [27]:
g.solution.values

array([35000.,  3000.,  4000.,     0.])

In [28]:
results = dict(zip(g.solution.fuel_type.values, g.solution.values))
results

{'Coal': 35000.0, 'Wind': 3000.0, 'Gas': 4000.0, 'Oil': 0.0}

### Two bidding zones with transmission

Let's add a spatial dimension, such that the optimisation problem is expanded to
\begin{equation}
  \min_{g_{i,s}, f_\ell} \sum_s o_{i,s} g_{i,s}
\end{equation}
such that
\begin{align}
  g_{i,s} &\leq G_{i,s} \\
  g_{i,s} &\geq 0 \\
  \sum_s g_{i,s} - \sum_\ell K_{i\ell} f_\ell &= d_i & \text{KCL} \\
  |f_\ell| &\leq F_\ell & \text{line limits}  \\
  \sum_\ell C_{\ell c} x_\ell f_\ell &= 0 & \text{KVL} 
\end{align}

In this example, we connect the previous South African electricity system with a hydro generation unit in Mozambique through a single transmission line. Note that because a single transmission line will not result in any cycles, we can neglect KVL in this case.

We are given the following data (all in MW):

In [29]:
capacities = {
    "South Africa": {"Coal": 35000, "Wind": 3000, "Gas": 8000, "Oil": 2000,"Hydro": 0},
    "Mozambique": {"Coal": 0, "Wind": 0, "Gas": 0, "Oil": 0,"Hydro": 1200},
}

In [30]:
transmission = 500

In [31]:
loads = {"South Africa": 42000, "Mozambique": 650}

In [32]:
marginal_costs = {
    "Wind": 0,
    "Coal": 30,
    "Gas": 60,
    "Oil": 80,
    "Hydro": 0,
}

In [33]:
m = Model()

In [34]:
capacities["South Africa"].keys()

dict_keys(['Coal', 'Wind', 'Gas', 'Oil', 'Hydro'])

In [35]:
capacities["Mozambique"].keys()

dict_keys(['Coal', 'Wind', 'Gas', 'Oil', 'Hydro'])

In [36]:
fuel_type = pd.Index(capacities["South Africa"].keys(), name='fuel_type')
fuel_type

Index(['Coal', 'Wind', 'Gas', 'Oil', 'Hydro'], dtype='object', name='fuel_type')

In [37]:
capacities_series_south_africa = pd.Series(capacities["South Africa"].values(),index=capacities["South Africa"].keys())
capacities_series_south_africa

Coal     35000
Wind      3000
Gas       8000
Oil       2000
Hydro        0
dtype: int64

In [38]:
g_south_africa = m.add_variables(lower=0, upper=capacities_series_south_africa, coords=[fuel_type], name='g_south_africa')
g_south_africa

Variable (fuel_type: 5)
-----------------------
[Coal]: g_south_africa[Coal] ∈ [0, 3.5e+04]
[Wind]: g_south_africa[Wind] ∈ [0, 3000]
[Gas]: g_south_africa[Gas] ∈ [0, 8000]
[Oil]: g_south_africa[Oil] ∈ [0, 2000]
[Hydro]: g_south_africa[Hydro] ∈ [0, 0]

In [39]:
capacities_series_mozambique = pd.Series(capacities["Mozambique"].values(),index=capacities["Mozambique"].keys())
capacities_series_mozambique

Coal        0
Wind        0
Gas         0
Oil         0
Hydro    1200
dtype: int64

In [40]:
g_mozambique = m.add_variables(lower=0, upper=capacities_series_mozambique, coords=[fuel_type], name='g_mozambique')
g_mozambique

Variable (fuel_type: 5)
-----------------------
[Coal]: g_mozambique[Coal] ∈ [0, 0]
[Wind]: g_mozambique[Wind] ∈ [0, 0]
[Gas]: g_mozambique[Gas] ∈ [0, 0]
[Oil]: g_mozambique[Oil] ∈ [0, 0]
[Hydro]: g_mozambique[Hydro] ∈ [0, 1200]

In [41]:
marginal_costs_new = {k : marginal_costs[k] for k in capacities["South Africa"].keys()}
marginal_costs_new

{'Coal': 30, 'Wind': 0, 'Gas': 60, 'Oil': 80, 'Hydro': 0}

In [42]:
marginal_costs_series= pd.Series(marginal_costs_new.values(),index=marginal_costs_new.keys())
marginal_costs_series

Coal     30
Wind      0
Gas      60
Oil      80
Hydro     0
dtype: int64

In [43]:
line_flow=m.add_variables(lower=-1*transmission, upper=transmission, name='line_flow')
line_flow

Variable
--------
line_flow ∈ [-500, 500]

In [44]:
g_south_africa.sum() - line_flow == loads["South Africa"]

Constraint (unassigned)
-----------------------
+1 g_south_africa[Coal] + 1 g_south_africa[Wind] + 1 g_south_africa[Gas] + 1 g_south_africa[Oil] + 1 g_south_africa[Hydro] - 1 line_flow = 42000

In [45]:
m.add_constraints(g_south_africa.sum() - line_flow == loads["South Africa"], name='total-generation-constraint-south-africa')

Constraint `total-generation-constraint-south-africa`
-----------------------------------------------------
+1 g_south_africa[Coal] + 1 g_south_africa[Wind] + 1 g_south_africa[Gas] + 1 g_south_africa[Oil] + 1 g_south_africa[Hydro] - 1 line_flow = 42000

In [46]:
g_mozambique.sum() + line_flow == loads["Mozambique"]

Constraint (unassigned)
-----------------------
+1 g_mozambique[Coal] + 1 g_mozambique[Wind] + 1 g_mozambique[Gas] + 1 g_mozambique[Oil] + 1 g_mozambique[Hydro] + 1 line_flow = 650

In [47]:
m.add_constraints(g_mozambique.sum() + line_flow == loads["Mozambique"], name='total-generation-constraint-mozambique')

Constraint `total-generation-constraint-mozambique`
---------------------------------------------------
+1 g_mozambique[Coal] + 1 g_mozambique[Wind] + 1 g_mozambique[Gas] + 1 g_mozambique[Oil] + 1 g_mozambique[Hydro] + 1 line_flow = 650

In [48]:
obj = (g_south_africa*marginal_costs_series).sum()+(g_mozambique*marginal_costs_series).sum()

m.add_objective(obj)

LinearExpression
----------------
+30 g_south_africa[Coal] + 0 g_south_africa[Wind] + 60 g_south_africa[Gas] ... +60 g_mozambique[Gas] + 80 g_mozambique[Oil] + 0 g_mozambique[Hydro]

In [49]:
m.solve()

Restricted license - for non-production use only - expires 2024-10-28
Read LP format model from file C:\Users\huang\AppData\Local\Temp\linopy-problem-k2kqdgcv.lp
Reading time = 0.00 seconds
obj: 2 rows, 11 columns, 12 nonzeros
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 11 columns and 12 nonzeros
Model fingerprint: 0xa36f3a24
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 8e+01]
  Bounds range     [5e+02, 4e+04]
  RHS range        [7e+02, 4e+04]
Presolve removed 2 rows and 11 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2600000e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  

('ok', 'optimal')

In [50]:
g_south_africa.solution.fuel_type.values

array(['Coal', 'Wind', 'Gas', 'Oil', 'Hydro'], dtype=object)

In [51]:
g_south_africa.solution.values

array([35000.,  3000.,  3500.,     0.,     0.])

In [52]:
results_south_africa = dict(zip(g_south_africa.solution.fuel_type.values, g_south_africa.solution.values))
results_south_africa

{'Coal': 35000.0, 'Wind': 3000.0, 'Gas': 3500.0, 'Oil': 0.0, 'Hydro': 0.0}

In [53]:
g_mozambique.solution.fuel_type.values

array(['Coal', 'Wind', 'Gas', 'Oil', 'Hydro'], dtype=object)

In [54]:
g_mozambique.solution.values

array([   0.,    0.,    0.,    0., 1150.])

In [55]:
results_mozambique = dict(zip(g_mozambique.solution.fuel_type.values, g_mozambique.solution.values))
results_mozambique

{'Coal': 0.0, 'Wind': 0.0, 'Gas': 0.0, 'Oil': 0.0, 'Hydro': 1150.0}